## 하이퍼 파라미터

In [8]:
import tensorflow as tf
import numpy as np

In [9]:
EPOCHS = 10

## 모델 정의

In [2]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        conv2d = tf.keras.layers.Conv2D
        maxpool = tf.keras.layers.MaxPool2D
        self.sequence = list() #
        self.sequence.append(conv2d(16, (3, 3), padding='same', activation='relu')) # Padding이 Valid 형태 -> Zero Padding을 해주지 않아 Conv layer를 통과하면서 영상 크기가 줄기 때문에 same으로
        self.sequence.append(conv2d(16, (3, 3), padding='same', activation='relu')) # 28x28x16
        self.sequence.append(maxpool((2,2))) # 14x14x16
        self.sequence.append(conv2d(32, (3, 3), padding='same', activation='relu')) #14x14x32
        self.sequence.append(conv2d(32, (3, 3), padding='same', activation='relu')) #14x14x32
        self.sequence.append(maxpool((2,2))) # 7x7x16
        self.sequence.append(conv2d(64, (3, 3), padding='same', activation='relu')) #7x7x64
        self.sequence.append(conv2d(64, (3, 3), padding='same', activation='relu')) #7x7x64
        self.sequence.append(tf.keras.layers.Flatten()) # 1568
        self.sequence.append(tf.keras.layers.Dense(2048, activation='relu'))
        self.sequence.append(tf.keras.layers.Dense(10, activation='softmax'))
        
        
    def call(self, x, training=False, mask=None):
        for layer in self.sequence:
            x = layer(x)
        return x

## 학습 테스트 루프 정의

In [3]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
    
# Implement algorithm test
def  test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test /255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

11493376/11490434 [==============================] - 1s 0us/step


## 학습 환경 정의

#### 모델 생성, 손실 함수, 최적화 알고리즘, 평가지표 정의

In [5]:
# Create Model
model = ConvNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [10]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
    
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
    
    template = ' Epoch {}, Loss: {}, Accuracy: {}, Test Loss : {}, Test Accuracy : {}'
    print(template.format(epoch + 1,
                         train_loss.result(),
                         train_accuracy.result() * 100,
                         test_loss.result(),
                         test_accuracy.result() * 100))

 Epoch 1, Loss: 0.10762634128332138, Accuracy: 96.65499877929688, Test Loss : 0.05623356252908707, Test Accuracy : 98.44999694824219
 Epoch 2, Loss: 0.07599354535341263, Accuracy: 97.65916442871094, Test Loss : 0.04702622815966606, Test Accuracy : 98.625
 Epoch 3, Loss: 0.06212647631764412, Accuracy: 98.086669921875, Test Loss : 0.04527907818555832, Test Accuracy : 98.72333526611328
 Epoch 4, Loss: 0.05313558503985405, Accuracy: 98.36833190917969, Test Loss : 0.04360963776707649, Test Accuracy : 98.79750061035156
 Epoch 5, Loss: 0.04734637588262558, Accuracy: 98.55033111572266, Test Loss : 0.042164094746112823, Test Accuracy : 98.83999633789062
 Epoch 6, Loss: 0.04276557266712189, Accuracy: 98.69249725341797, Test Loss : 0.04163222387433052, Test Accuracy : 98.86000061035156
 Epoch 7, Loss: 0.039201218634843826, Accuracy: 98.80142974853516, Test Loss : 0.040377579629421234, Test Accuracy : 98.87999725341797
 Epoch 8, Loss: 0.036459580063819885, Accuracy: 98.88541412353516, Test Loss : 